In [1]:
state = 'NE' 
year = 2020

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json
from epsilon_constraint import*
import networkx as nx

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of NE has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from NE_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [6]:
obj_type = 'cut_edges'
starting_deviation = 0.01 
verbose = True
contiguity = 'lcut'               # {'lcut', 'scf', 'shir'}
warm_start_mode = 'user'          # {'None', 'user', 'refinement'}
warm_starts = warm_starts         # if you have user define warm starts else it is None
time_limit = 7200                 # Time limit in seconds, adjust as needed
multiplier = 0                    # Adjust multiplier as needed
max_B = True                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True
symmetry_breaking = 'orbitope'    # {None, 'orbitope', 'rsum'}              
sizes = None

In [ ]:
print(f"Running epsilon constraint method for state {state}, objective {obj_type}")
print(f"{'#' * 100}\n")
(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity,        
            cutoff=None,       
            verbose=verbose,
            warm_start_mode = warm_start_mode,
            warm_starts=warm_starts,  
            starting_deviation=starting_deviation, 
            time_limit=time_limit, 
            sizes=sizes,      
            max_B=max_B,      
            symmetry_breaking=symmetry_breaking, 
        )

Running epsilon constraint method for state NE, objective cut_edges
####################################################################################################

L = 647297 and U = 660373

****************************************
Trying deviation = 6538.346666666666
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 647297 and U = 660373
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (

Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 3804 rows, 3243 columns and 12403 nonzeros
Model fingerprint: 0x686f7f04
Variable types: 837 continuous, 2406 integer (2406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 1965 row

Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 3804 rows, 3243 columns and 12403 nonzeros
Model fingerprint: 0xb43763db
Variable types: 837 continuous, 2406 integer (2406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 1954 row

  Lazy constraints: 49

Explored 3064 nodes (301567 simplex iterations) in 36.94 seconds (5.51 work units)
Thread count was 20 (of 20 available processors)

Solution count 3: 22 23 32 

Optimal solution found (tolerance 0.00e+00)
Best objective 2.200000000000e+01, best bound 2.200000000000e+01, gap 0.0000%

User-callback calls 6941, time in user-callback 7.89 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[9, 12, 26, 32, 52, 76, 80], [6, 7, 10, 28, 33, 47, 48, 60, 61, 65, 69, 71, 87, 91], [0, 1, 2, 3, 4, 5, 8, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 66, 67, 68, 70, 72, 73, 74, 75, 77, 78, 79, 81, 82, 83, 84, 85, 86, 88, 89, 90, 92]]

****************************************
Trying deviation = 690.4999999999612
****************************************
No valid warm start u

     0     0   13.20378    0  364          -   13.20378      -     -    7s
     0     0   13.20465    0  376          -   13.20465      -     -    7s
     0     0   13.20476    0  376          -   13.20476      -     -    7s
     0     0   13.20585    0  376          -   13.20585      -     -    7s
     0     0   13.20585    0  377          -   13.20585      -     -    7s
     0     0   13.21782    0  365          -   13.21782      -     -    7s
     0     0   13.21800    0  368          -   13.21800      -     -    8s
     0     0   13.22247    0  372          -   13.22247      -     -    8s
     0     0   13.22289    0  368          -   13.22289      -     -    9s
     0     0   13.22347    0  370          -   13.22347      -     -    9s
     0     0   13.22347    0  369          -   13.22347      -     -    9s
     0     2   13.22381    0  369          -   13.22381      -     -   10s
   471   465   33.99136   25   65          -   14.41805      -   110   15s
  1219  1076 infeasible  

Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 3804 rows, 3243 columns and 12403 nonzeros
Model fingerprint: 0xdae32f94
Variable types: 837 continuous, 2406 integer (2406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 1993 rows and 1411 columns
Presolve time: 0.13s
Presolved: 1811 rows, 1832 columns, 6307 nonzeros
Variable types: 0 continuous, 1832 integer (1832 binary)

Root relaxation: objective 1.175088e+01, 2017 iterations, 0.44 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.75088    0  255          -   11.75088      -     -    1s
     0 

     0     2   11.08188    0  347          -   11.08188      -     -   12s
   130   150   20.70664   11  211          -   14.33383      -   118   15s
   870   852   49.60522   45   30          -   14.33383      -  78.1   20s
  1840  1602   23.76257   42  349          -   14.55385      -  80.0   25s
  1847  1607   39.37093   68  466          -   14.55385      -  79.7   30s
  1857  1616   14.55385   14  254          -   14.55385      -  82.1   35s
  2116  1822   28.52129   23  292          -   14.91082      -  93.1   40s
  2558  2093   28.80941   30  384          -   14.94496      -  97.0   45s
  3287  2496   40.99429   42  122          -   15.36700      -  94.8   50s
  4490  3026   45.99833   46  269          -   15.36700      -  91.2   55s
  5964  3628   29.58932   36  153          -   15.82135      -  82.5   60s
  8204  4969 infeasible   37               -   15.86581      -  78.3   65s
 10053  6459   37.30149   41  114          -   16.99804      -  75.1   70s
 11294  7366   32.03212  

  4542  2966   33.20277   36  234          -   16.01696      -  86.8   65s
  5492  3239   23.97621   31  199          -   16.01696      -  85.0   70s
  6774  3943   24.39958   23  310          -   16.59792      -  82.2   75s
  7903  4623   39.69892   43   30          -   16.69048      -  81.0   80s
  9814  5627   48.69002   40  149          -   17.61595      -  80.1   85s
 11065  6606   28.50839   35   53          -   20.42587      -  80.4   90s
 12487  7466   49.03162   69   93          -   20.46962      -  79.8   96s
 13467  8253   31.67455   38  272          -   20.47225      -  79.0  100s
 15044  9172   38.59091   34  300          -   20.47225      -  79.7  106s
 17020 10488   31.86132   36  288          -   20.55411      -  79.6  110s
 18415 11066   41.35815   39  415          -   22.55343      -  80.4  115s
 19540 11810   26.21074   56   60          -   22.96608      -  80.9  120s
 21848 13261 infeasible   32               -   23.60958      -  82.1  126s
 23595 13946   29.68647  

  Inf proof: 2
  Zero half: 5
  RLT: 5
  Lazy constraints: 355

Explored 30550 nodes (2329557 simplex iterations) in 160.68 seconds (22.89 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 34 

Optimal solution found (tolerance 0.00e+00)
Best objective 3.400000000000e+01, best bound 3.400000000000e+01, gap 0.0000%

User-callback calls 64090, time in user-callback 16.82 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[9, 10, 26, 43, 46, 76], [6, 7, 20, 21, 28, 37, 47, 60, 65, 69, 71, 72, 87, 91], [0, 1, 2, 3, 4, 5, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 66, 67, 68, 70, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 92]]

****************************************
Trying deviation = 151.16666666670548
*********************

 57268 13713   39.91838   31  191   45.00000   29.14943  35.2%  87.2  348s
 58934 13908   30.70571   38   83   45.00000   29.27989  34.9%  87.9  357s
 60625 13906 infeasible   43        45.00000   29.80862  33.8%  88.3  369s
 60693 13976   32.92219   51  171   45.00000   29.80862  33.8%  88.4  381s
 61799 14099   43.16827   39   18   45.00000   29.91352  33.5%  88.8  391s
 63537 14188   40.64739   43  376   45.00000   30.36083  32.5%  89.3  402s
 65313 14246     cutoff   28        45.00000   30.68470  31.8%  89.9  411s
 67237 14330   43.12832   33  255   45.00000   31.18928  30.7%  90.3  424s
 69137 14363   39.44075   50  254   45.00000   31.41377  30.2%  90.7  437s
 71133 14530   35.63440   29  334   45.00000   31.87389  29.2%  91.0  448s
 73189 14537   42.85484   37  242   45.00000   32.14024  28.6%  91.1  459s
 75415 14518 infeasible   68        45.00000   32.45843  27.9%  91.1  470s


In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
if any(round(x[1][0], 2) != round(x[1][1], 2) for x in result):
    min_deviation = max(x[2] for x in result if round(x[1][0], 2) != round(x[1][1], 2))
else:
    min_deviation = min(x[2] for x in result)
    
print('min_deviation:' , min_deviation)

In [ ]:
from enumeration import enumerate_and_solve_k2_subproblems
from metrics import *
import math

epsilon = 1 / (2 * G._k)

while True:
    print(f"\n{'*' * 40}\nTrying deviation = {min_deviation}\n{'*' * 40}")
    L = math.ceil(G._ideal_population - min_deviation)
    U = math.floor(G._ideal_population + min_deviation)
    G._L = L
    G._U = U
    print(f'L = {L} and U = {U}')

    new_plans = enumerate_and_solve_k2_subproblems(G, min_deviation, L, U, G._k, obj_type = obj_type, root='Douglas')
    
    if not new_plans:
        print("No plans found. Reducing deviation or stopping.")
        break  

    enumeration_plans = {}
    for i, plan in enumerate(new_plans):
        obj_value = compute_obj(G, plan, obj_type)
        dev = observed_deviation_persons(G, plan, G._ideal_population, year=year)
        enumeration_plans[i] = [obj_value, dev]

    if obj_type in {"inverse_Polsby_Popper", "cut_edges", "perimeter"}:
        best_obj_value = min(val[0] for val in enumeration_plans.values())
    else:
        best_obj_value = max(val[0] for val in enumeration_plans.values())
    print(f"best_obj_value {obj_type}: {best_obj_value}")

    best_plan_index = None
    best_dev = float('inf')

    for i, (val, dev) in enumeration_plans.items():
        if val == best_obj_value and dev < best_dev:
            best_plan_index = i
            best_dev = dev

    if best_plan_index is not None:
        best_entry = (new_plans[best_plan_index], [best_obj_value, best_obj_value], best_dev)

        # Check if an entry with the same deviation already exists
        existing_index = next((i for i, entry in enumerate(result) if math.isclose(entry[2], best_dev)), None)

        if existing_index is not None:
            result[existing_index] = best_entry
            print(f"Replaced plan at index {existing_index} with new plan {best_plan_index} (dev={best_dev}, {best_obj_value} {obj_type})")
        else:
            result.append(best_entry)
            print(f"Added plan {best_plan_index} with {best_obj_value} {obj_type} and deviation {best_dev} to result")

    if best_dev < epsilon:
        print("Deviation is too small, stopping early.")
        break

    min_deviation = best_dev - epsilon

In [ ]:
# Determine the senses for the Pareto frontier
# Minimize the first objective ('deviation').
# Maximize the second objective for compactness types, otherwise minimize it.
senses = ['min', 'max' if obj_type in ['average_Polsby_Popper','bottleneck_Polsby_Popper'] else 'min']
obj_names = ['deviation_persons', obj_type]
infeasible_region = [0, min(round(r[2],1) for r in result)]

In [ ]:
from pareto import ParetoFrontier

print(f"\n{'#' * 100}\nPareto Frontier for state {state},  objective {obj_type}\n{'#' * 100}\n")
pareto = ParetoFrontier(senses, obj_names, state=state, level='county')

if not  plans:
    print("No plan found!")
else:
    for plan, obj_bound, dev in result:
        upper_bound = [dev,  1/obj_bound[0] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[0]]
        lower_bound = [dev,  1/obj_bound[1] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[1]]
        
        pareto.add_plan(plan, upper_bound, lower_bound)
            
print("Pareto front plans:", pareto.plans)
print("Pareto front upper bounds:", pareto.upper_bounds)
print("Pareto front lower bounds:", pareto.lower_bounds) 

extra_points = None
extra_colors = None
        
pareto.tighten_lower_bounds()

if infeasible_region[1] > 0.5:
    max_dev = max(upper_bound[0] for upper_bound in pareto.upper_bounds)
    max_obj = max(upper_bound[1] for upper_bound in pareto.upper_bounds)
    min_obj = min(upper_bound[1] for upper_bound in pareto.upper_bounds)
    o1lim=[-100, max_dev+100]
    o2lim=[min_obj*0.9, max_obj*(1.1)]
else:
    o1lim = None
    o2lim = None

pareto.plot_with_custom_x_ranges(method = 'epsilon_constraint_method', splits=None,
                                 o1lim=o1lim, o2lim=o2lim, infeasible_region = infeasible_region,
                                 extra_points=extra_points, extra_colors=extra_colors)

In [ ]:
from draw import *

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: round(x, 0)}

for plan, obj_bound, dev in result:
    ub = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[0])
    lb = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[1])
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    if ub != lb:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), obj_bounds [{ub}, {lb}] {obj_type}"
    else:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), with {ub} {obj_type}"

    draw_plan(filepath, filename2, G, plan, title=title, year=year)